## Parameter pair correlation
This notebook produces 3D plots of user-specified pairs of parameters from the algorithm.

In [ ]:
import arviz as az
import numpy as np
from itertools import combinations

from aust_covid.constants import RUN_IDS, PRIMARY_ANALYSIS, BURN_IN, set_project_base_path
from emutools.utils import load_param_info
from emutools.calibration import plot_3d_param_hist, plot_3d_param_corr

project_paths = set_project_base_path("../")
RUNS_PATH = project_paths["RUNS_PATH"]
OUTPUTS_PATH = project_paths["OUTPUTS_PATH"]

In [ ]:
idata = az.from_netcdf(RUNS_PATH / RUN_IDS[PRIMARY_ANALYSIS] / "output/calib_full_out.nc")
idata = idata.sel(draw=np.s_[BURN_IN:])

Available parameters for comparison are:

In [ ]:
list(idata.posterior.keys())

In [ ]:
key_params = [
    "contact_rate",
    "start_cdr",
    "infectious_period",
    "imm_prop",
    "natural_immunity_period",
]

In [ ]:
for params in combinations(key_params, 3):
    fig = plot_3d_param_corr(idata, params, marker_size=3, n_samples=5000)
    fig.write_html(OUTPUTS_PATH / f"corr_{params[0]}X{params[1]}X{params[2]}.html")
fig

In [ ]:
# Generating text for linking repo to online interactive visuals
# abbreviations = load_param_info()["abbreviations"]
# for params in combinations(key_params, 3):
    # print(f"- [{abbreviations[params[0]]} versus {abbreviations[params[1]]} versus {abbreviations[params[2]]}](corr_{params[0]}X{params[1]}X{params[2]})")